In [1]:
import pandas as pd

movies = pd.read_csv("data/movies.csv", index_col='id')

movies[['genres_id', 'actors_id', 'directors_id']] = movies[[
    'genres_id', 'actors_id', 'directors_id']].apply(lambda x: x.apply(eval))

genres = pd.read_csv("data/genres.csv", index_col='genre').to_dict(
    orient='dict')['id']
actors = pd.read_csv("data/actors.csv", index_col='actor').to_dict(
    orient='dict')['id']
directors = pd.read_csv("data/directors.csv", index_col='director').to_dict(
    orient='dict')['id']


In [2]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 862 to 461257
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   budget         45345 non-null  float64
 1   popularity     45345 non-null  float64
 2   release_date   45345 non-null  object 
 3   revenue        45345 non-null  float64
 4   runtime        45345 non-null  float64
 5   title          45345 non-null  object 
 6   vote_average   45345 non-null  float64
 7   vote_count     45345 non-null  float64
 8   genres_id      45345 non-null  object 
 9   actors_id      45345 non-null  object 
 10  directors_id   45345 non-null  object 
 11  release_year   45345 non-null  int64  
 12  release_month  45345 non-null  int64  
 13  release_day    45345 non-null  object 
 14  return         45345 non-null  float64
dtypes: float64(7), int64(2), object(6)
memory usage: 5.5+ MB


In [79]:
#Primera función 
def cantidad_filmaciones_mes(mes: str):
    mes = mes.lower().strip()
    numero_mes = mes_to_numero(mes)

    cantidad = len(movies[movies['release_month'] == numero_mes])

    return {'mes': mes, 'cantidad': cantidad}

def mes_to_numero(mes: str) -> int:
    meses = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
    }

    return meses.get(mes, 0)

In [80]:
cantidad_filmaciones_mes("abril")

{'mes': 'abril', 'cantidad': 3452}

In [67]:
#Segunda función 
def cantidad_filmaciones_dia(dia: str):
    dia = dia.lower().strip()
    cantidad = len(movies[movies['release_day'].str.lower() == dia])
    
    return {'dia': dia, 'cantidad': cantidad}

In [71]:
cantidad_filmaciones_dia("lunes")

{'dia': 'lunes', 'cantidad': 3500}

In [83]:
#Tercera función 
def score_titulo(titulo_de_la_filmacion: str):
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower().strip()
    filmacion = movies[movies['title'].str.lower() == titulo_de_la_filmacion]
    
    if filmacion.empty:
        return "No se encontró la filmación especificada."
    else:
        titulo = filmacion['title'].values[0]
        año_estreno = filmacion['release_year'].values[0]
        score = filmacion['popularity'].values[0]
        
        return {'titulo': titulo, 'anio': año_estreno, 'popularidad': score}


In [84]:
score_titulo("A Little Princess")

{'titulo': str, 'anio': 1995, 'popularidad': 7.731725}

In [74]:
# Cuarta función 
def votos_titulo(titulo_de_la_filmacion: str):
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower().strip()
    filmacion = movies[movies['title'].str.lower() == titulo_de_la_filmacion]
    
    if filmacion.empty:
        return "No se encontró la filmación especificada."
    
    vote_count = filmacion['vote_count'].values[0]
    vote_average = filmacion['vote_average'].values[0]
    release_year = filmacion['release_year'].values[0]
    
    if vote_count < 2000:
        return "La filmación no cumple con el requisito mínimo de 2000 valoraciones."
    
    return {'titulo': titulo_de_la_filmacion, 'anio': release_year, 'voto_total': vote_count, 'voto_promedio': vote_average}

In [75]:
votos_titulo("Star Wars")

{'titulo': 'star wars',
 'anio': 1977,
 'voto_total': 6778.0,
 'voto_promedio': 8.1}

In [77]:
#Quinta función
def get_actor(nombre_actor: str):
    nombre_actor = nombre_actor.lower().strip()
    
    if nombre_actor in actors:
        actor_id = actors[nombre_actor]
        participaciones = sum(movies['actors_id'].apply(lambda x: actor_id in x))
        retorno_total = movies[movies['actors_id'].apply(lambda x: actor_id in x)]['return'].sum()
        promedio_retorno = retorno_total / participaciones if participaciones > 0 else 0
        
        return {'actor': nombre_actor, 'cantidad_filmaciones': participaciones, 'retorno_total': retorno_total, 'retorno_promedio': promedio_retorno}
    else:
        return f"No se encontró el actor {nombre_actor} en el dataset."

In [78]:
get_actor("tom hanks")

{'actor': 'tom hanks',
 'cantidad_filmaciones': 71,
 'retorno_total': 178.8494960552024,
 'retorno_promedio': 2.5190069866929914}

In [62]:
# Sexta función

def get_director(nombre_director: str):
    nombre_director = nombre_director.lower().strip()
    
    if nombre_director in directors:
        director_id = directors[nombre_director]
        peliculas_director = movies[movies['directors_id'].apply(lambda x: director_id in x)]
        
        if peliculas_director.empty:
            return {"director": nombre_director, "películas": [], "retorno_total_director": 0}
        
        retorno_total = peliculas_director['return'].sum()
        
        resultado = {
            "director": nombre_director,
            "retorno_total_director": retorno_total,
            "películas": []
        }
        
        for index, pelicula in peliculas_director.iterrows():
            titulo = pelicula['title']
            fecha_lanzamiento = pelicula['release_date']
            retorno = pelicula['return']
            costo = pelicula['budget']
            ganancia = pelicula['revenue']
            
            pelicula_info = {
                "Título": titulo,
                "Fecha_lanzamiento": fecha_lanzamiento,
                "retorno": retorno,
                "presupuesto": costo,
                "recaudación": ganancia
            }
            
            resultado["películas"].append(pelicula_info)
        
        return resultado
    else:
        return {"director": nombre_director, "películas": [], "retorno_total_director": 0}

In [63]:
get_director("Oliver stone")

{'director': 'oliver stone',
 'retorno_total_director': 58.012846160040326,
 'películas': [{'Título': 'Nixon',
   'Fecha_lanzamiento': '1995-12-22',
   'retorno': 0.3109492045454545,
   'presupuesto': 44000000.0,
   'recaudación': 13681765.0},
  {'Título': 'Natural Born Killers',
   'Fecha_lanzamiento': '1994-08-25',
   'retorno': 1.4789048823529412,
   'presupuesto': 34000000.0,
   'recaudación': 50282766.0},
  {'Título': 'Heaven & Earth',
   'Fecha_lanzamiento': '1993-12-24',
   'retorno': 0.0,
   'presupuesto': 33000000.0,
   'recaudación': 0.0},
  {'Título': 'Platoon',
   'Fecha_lanzamiento': '1986-12-18',
   'retorno': 23.0884275,
   'presupuesto': 6000000.0,
   'recaudación': 138530565.0},
  {'Título': 'The Doors',
   'Fecha_lanzamiento': '1991-03-01',
   'retorno': 0.9057077105263158,
   'presupuesto': 38000000.0,
   'recaudación': 34416893.0},
  {'Título': 'U Turn',
   'Fecha_lanzamiento': '1997-10-03',
   'retorno': 0.3516893684210526,
   'presupuesto': 19000000.0,
   'recauda